In [ ]:
%pip install --upgrade pip
%pip install python2verilog[full]

In [ ]:
import matplotlib.pyplot as plt
import random
import numpy as np
from typing import Optional
from python2verilog import verilogify, Modes, context_to_verilog
from python2verilog.utils import make_visual


In [ ]:
namespace = {}


@verilogify(
    namespace=namespace,
    mode=Modes.OVERWRITE,
)
def draw_circle(centre_x, centre_y, radius):
    offset_y = 0
    offset_x = radius
    crit = 1 - radius
    while offset_y <= offset_x:
        yield (centre_x + offset_x, centre_y + offset_y)  # -- octant 1
        yield (centre_x + offset_y, centre_y + offset_x)  # -- octant 2
        yield (centre_x - offset_x, centre_y + offset_y)  # -- octant 4
        yield (centre_x - offset_y, centre_y + offset_x)  # -- octant 3
        yield (centre_x - offset_x, centre_y - offset_y)  # -- octant 5
        yield (centre_x - offset_y, centre_y - offset_x)  # -- octant 6
        yield (centre_x + offset_x, centre_y - offset_y)  # -- octant 8
        yield (centre_x + offset_y, centre_y - offset_x)  # -- octant 7
        offset_y = offset_y + 1
        if crit <= 0:
            crit = crit + 2 * offset_y + 1
        else:
            offset_x = offset_x - 1
            crit = crit + 2 * (offset_y - offset_x) + 1


for _ in range(3):
    x, y, radius = random.randint(10, 30), random.randint(10, 30), random.randint(1, 10)
    print(f"x: {x}, y: {y}, radius: {radius}")
    make_visual(draw_circle(x, y, radius))

module, testbench = context_to_verilog(namespace[draw_circle.__name__])

print(module)
print(testbench)
